<a href="https://colab.research.google.com/github/scmosoeu/fresh_produce/blob/master/Woolworths_fresh_produce_web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Woolworths Fresh Produce Webscraper

## Importing Libraries


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from urllib.request import urlopen, URLError, Request
import re
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import requests
from datetime import date
from sqlalchemy import create_engine
import pyodbc

### Webscrape with BeautifulSoup

In [ ]:
url = 'https://www.woolworths.co.za/cat/Food/Fruit-Vegetables-Salads/_/N-lllnam'

response = urlopen(url)
page = response.read()

woolworths_soup = BeautifulSoup(page, 'html.parser')



In [ ]:
woolworths_soup.find_all('div', {'class':'product--image'})

[]

In [ ]:
woolworths_soup.find_all('script')

[<script type="text/javascript">(function(){(function(){function a(){if(void 0===b.dialogArguments)return navigator.cookieEnabled;document.cookie="__dTCookie=1";var a=-1!==document.cookie.indexOf("__dTCookie");document.cookie="__dTCookie=1; expires=Thu, 01-Jan-1970 00:00:01 GMT";return a}if(window.dT_)window.console&&window.console.log("Duplicate agent injection detected, turning off redundant initConfig.");else{var b="undefined"!==typeof window?window:self;a()&&(window.dT_||(window.dT_={cfg:"async=1|bandwidth=900|md=CustomerName=aspan.text-xsmall.main-header-nav__label,OrderConfirmationText=aorder-confirm-page.grid.content--centered|lastModification=1593765877456|tp=500,50,0,1|reportUrl=dynaTraceMonitor|agentUri=/dtagent_A23Vbhjpqrx_7000200231032.js|auto=1|domain=woolworths.co.za|rid=RID_-1768944640|rpid=186470704|app=Woolworths Online",
 iCE:a}))}})();}).call(this);
 
 
 (function(){(function(){function Mb(){return ta?new ta:ua?new ua("MSXML2.XMLHTTP.3.0"):d.XMLHttpRequest?new d.XMLH

### Webscrape Dynamic site with Webdriver

In [ ]:
# please ensure you install geckodriver and change your path as shown here (https://towardsdatascience.com/data-science-skills-web-scraping-javascript-using-python-97a29738353f)

options = Options()
# Set headless=True to cut on compute time
options.headless = True
driver = webdriver.Firefox(options=options)
# Pass URL
driver.get(url)
# assign page to html
html = driver.page_source
driver.quit()
# apply BeautifulSoup to html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
products = soup.find_all('article', {'class':'product-card'})

In [ ]:
# to scrape the relevant data from the link 
def woolworths_web_scraper(link):
    """
    
    """
    
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options)
    driver.get(link)
    html = driver.page_source
    driver.quit()
    soup = BeautifulSoup(html, 'lxml')
    
    products = soup.find_all('article', {'class':'product-card'})
    item_woolworths = []
    price_woolworths = []

    for product in products:
        item_woolworths.append(product.find('a', {'class':'range--title'}).text.strip())
        price_woolworths.append(product.find('strong', {'class':'price'}).text.strip('R '))
        

    if len(soup.find_all('a', {'class':'pagination__nav'})) > 1:
        link_str= str(soup.find_all('a', {'class':'pagination__nav'})[-1])
        pattern = '<a class="pagination__nav".*?>'
        next_page = re.search(pattern, link_str).group()
        next_page = next_page[len('<a class="pagination__nav" href=')+1: -2 ].split('"')[0]
        next_page = 'https://www.woolworths.co.za' + next_page
    
        

    woolworths_fresh_produce = pd.DataFrame({'Item':item_woolworths, 'Price':price_woolworths}, columns=['Item', 'Price'])
    
    woolworths_fresh_produce['Date'] = [date.today() for k in range(len(woolworths_fresh_produce))] 
    
    
    return woolworths_fresh_produce, next_page
    
    

In [ ]:
# Scrapping data from first page
woolworths_fresh_produce, next_page = woolworths_web_scraper(url)

In [ ]:
#iterating through the pages using the 'next' icon
links = []
while next_page not in links:
    links.append(next_page)
    df, next_page = woolworths_web_scraper(next_page)
    if df.empty:
        break
    else: 
        woolworths_fresh_produce = pd.concat([woolworths_fresh_produce, df])

woolworths_fresh_produce.reset_index(drop=True, inplace=True)    
woolworths_fresh_produce.tail(5)

,Item,Price,Date
545,Sliced Portabellini Mushrooms 250g,29.99,2020-08-28
546,Beetroot 1Kg,22.99,2020-08-28
547,Seedless Limes 1Kg,34.99,2020-08-28
548,Roasted Baby Marrow with Olive Oil 225g,29.99,2020-08-28
549,"Portobellini Mushrooms, Spinach & Red Onion 270g",29.99,2020-08-28


In [ ]:
woolworths_fresh_produce.to_csv('Woolworths_prices.csv', index=False)

In [ ]:
# engine = create_engine('sqlite://', echo=False)

# sql_conn =pyodbc.connect(driver='{SQL Server}',
#                       host='USER\SQLEXPRESS',
#                       database='Woolworths_fresh_produce',
#                       trusted_connection='true',
#                       user='sa')

# woolworths_fresh_produce.to_sql('Woolworths_prices', con=engine)

# query = "select * from Woolworths_prices"
# engine.execute(query).fetchall()
